In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(30)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

directory_out = './file/2.part'

if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

# = = = = = = = = = = = = = = =

directory_in = './file/1.part_number'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
for file in list_file:

    # = = = = = = = = = = = = = = =

    list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                           header=0,
                                                                           dtype=str).fillna('')

    df_input['No'] = df_input['No'].astype(int)

    count_remain = len(df_input)

    print(f'总数量：{count_remain}')
    print()

    # = = = = = = = = = = = = = = =

    def crawler(crawler_tuple):
        global list_dict_correct, list_series_error, count_remain

        crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

        # = = = = = = = = = = = = = = =

        try:
            count_retry = 0
            while True:
                count_retry += 1

                try:
                    url_request, url_check = f'''https://ecatalog.smpcorp.com/v2/fs/api/part/partselect?part={crawler_series['Part Number']}&func=PART&vid=''', crawler_series['Url']

                    resp = requests.get(url_request,
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(5, 15))

                    if resp.status_code == 200:
                        dict_dict = resp.json()

                        if dict_dict:
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            dict_info = {key.strip().title(): value.strip() if type(value) == str else value for key, value in dict_dict['pd'].items() if key not in ['rowId', 'site', 'webBase', 'basePart', 'maxRows'] and value != None}

            # = = = = = = = = = = = = = = =

            vehicle = '\n'.join([dict_['buyersGuideDesc'].strip() for dict_ in dict_dict['pp']['buyersGuides']])

            # = = = = = = = = = = = = = = =

            list_dict, dict_count = dict_dict['pp']['partSpecs'], {}
            for dict_ in list_dict:
                key = dict_['attributeName_en'].strip()
                if key in dict_count:
                    dict_count[key] += 1
                else:
                    dict_count[key] = 1

            dict_specification, dict_no = {}, {}
            for i, dict_ in enumerate(list_dict):
                key = dict_['attributeName_en'].strip()
                if dict_count[key] == 1:
                    dict_specification[str(i)] = {key: dict_['siteValue_en'].strip()}
                else:
                    if key in dict_no:
                        dict_no[key] += 1
                    else:
                        dict_no[key] = 1
                    dict_specification[str(i)] = {f'{key}-{dict_no[key]}': dict_['siteValue_en'].strip()}

            # = = = = = = = = = = = = = = =

            dict_type, list_dict = {}, dict_dict['pp']['categories']
            for i, dict_ in enumerate(list_dict):
                dict_type[str(i)] = dict_['categoryName_en'].strip()

            # = = = = = = = = = = = = = = =

            with semaphore_correct:
                list_dict_correct.append({'No': crawler_series['No'],
                                          'Part Number': crawler_series['Part Number'],
                                          'Vehicle': vehicle,
                                          'Picture': '',
                                          'Url': crawler_series['Url'],
                                          'Json_Type': json.dumps(dict_type),
                                          'Json_Info': json.dumps(dict_info),
                                          'Json_Specification': json.dumps(dict_specification)})

            # = = = = = = = = = = = = = = =

            crawler_status = True
        except:
            crawler_series['Request_Url'], crawler_series['Check_Url'] = url_request, url_check

            with semaphore_error:
                list_series_error.append(crawler_series)

        # = = = = = = = = = = = = = = =

        if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and count_remain == 1 or count_remain % 300 == 0 and count_remain != 0:
            clear_output()
        print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{file.removesuffix('.xlsx')}（{(list_file.index(file) + 1) / len(list_file) * 100:.2f}%）] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_request}\n''')

        with semaphore_remain:
            count_remain -= 1

    # = = = = = = = = = = = = = = =

    pool.map(crawler, df_input.iterrows())

    print('输出ing...')
    print()
    if list_dict_correct:
        df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                                 ascending=[True],
                                                                 ignore_index=True)
        df_correct.to_excel(f'''{directory_out}/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if list_series_error:
        df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                               ascending=[True],
                                                               ignore_index=True)
        df_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = [file for file in sorted(os.listdir(directory_out)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_out}/{file}')]
for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'{directory_out}/{file}',
              f'''{directory_out}/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：10

[True] - [请求1次] - [剩余9条] - [crawler_1（50.00%）] - [2024/10/29 18:10:31] - 7. https://ecatalog.smpcorp.com/v2/fs/api/part/partselect?part=44014&func=PART&vid=

[True] - [请求13次] - [剩余8条] - [crawler_1（50.00%）] - [2024/10/29 18:10:32] - 8. https://ecatalog.smpcorp.com/v2/fs/api/part/partselect?part=44015&func=PART&vid=

[True] - [请求11次] - [剩余7条] - [crawler_1（50.00%）] - [2024/10/29 18:10:32] - 5. https://ecatalog.smpcorp.com/v2/fs/api/part/partselect?part=44002&func=PART&vid=

[True] - [请求5次] - [剩余6条] - [crawler_1（50.00%）] - [2024/10/29 18:10:32] - 9. https://ecatalog.smpcorp.com/v2/fs/api/part/partselect?part=44017&func=PART&vid=

[True] - [请求20次] - [剩余5条] - [crawler_1（50.00%）] - [2024/10/29 18:10:32] - 10. https://ecatalog.smpcorp.com/v2/fs/api/part/partselect?part=44021&func=PART&vid=

[True] - [请求21次] - [剩余4条] - [crawler_1（50.00%）] - [2024/10/29 18:10:33] - 6. https://ecatalog.smpcorp.com/v2/fs/api/part/partselect?part=44003&func=PART&vid=

[True] - [请求18次] - [剩余3条] - [crawler_1（

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 2415.38it/s]

Done ~
